# 属性描述符

描述符是实现了特定协议的类。这里的特定协议指的是：__get__、__set__以及__delete__。描述符实现了对多个属性运用相同存储逻辑。上一章中的property类实现了完整的描述符协议，但是在实际使用过程中通常仅需要实现协议的一部分即可，大多数情况下仅实现__get__和__set__，甚至在一些只读的应用场合中仅实现__get__方法。

本章有一个断论："理解描述符是精通Python的关键"

## 属性验证

上一章中的特性工厂依赖于一个函数，即借助函数式编程模式避免重复编写读值方法以及设值方法。若使用面向对象的模式，则可以借助描述符类实现特性工厂类。

基础逻辑如下

首先有两个类：
* 托管类：需要创建某些属性的读值方法以及设值方法的类
* 描述符类：实现描述符协议的类

逻辑如下：
1. 描述符类会创建实例作为托管类的类属性
2. 描述符类会为需要托管的属性创建描述符实例，并且被托管的属性的读值以及设值会由描述符实例接管
3. 描述符实例不会存储被托管属性的值，被托管属性的值依然存储在托管类实例中（__dict__）

基于上述的定义以及逻辑，可以创建如下的托管类以及描述符类

In [7]:
class Quantity:

    def __init__(self, storage_name):
        self.storage_name = storage_name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError("value must be > 0")


class LineItem:

    weight = Quantity("weight")
    price = Quantity("price")

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


test_item = LineItem("test", 10, 1)
print("LineItem类属性")
print(test_item.__class__.__dict__)
print("\ntest_item实例属性")
print(test_item.__dict__)

LineItem类属性
{'__module__': '__main__', 'weight': <__main__.Quantity object at 0x0000020003B853C8>, 'price': <__main__.Quantity object at 0x0000020003B85188>, '__init__': <function LineItem.__init__ at 0x0000020003B935E8>, 'subtotal': <function LineItem.subtotal at 0x0000020003B93C18>, '__dict__': <attribute '__dict__' of 'LineItem' objects>, '__weakref__': <attribute '__weakref__' of 'LineItem' objects>, '__doc__': None}

test_item实例属性
{'description': 'test', 'weight': 10, 'price': 1}


上述例子中，LineItem实例的weight和price属性的设值方法分别被两个Quantity实例接管。

观察test_item的类属性以及实例属性可以发现：
1. test_item类属性中具有与被托管属性同名的实例
2. test_item实例属性中保存有被托管属性的值

当尝试对test_item中的weight以及price属性进行设值操作时，会优先尝试调用对应的特性。在上述例子中，会首先尝试调用类属性中对应描述符实例中的设值方法。当尝试对test_item中的weight以及price属性进行读值操作时，由于没有定义对应的特性，因此会直接返回实例属性中对应属性的值。

Quantity的__set__方法有三个参数：
1. self：描述符实例
2. instance：托管实例
3. value：尝试赋予的值

尤其需要注意的是：
Quantity的实例仅提供方法，**不能将被托管的属性的值存储到描述符实例中**。由于描述符实例是类属性，托管类所有的实例均会共享相同的描述符实例，若将被托管属性的值存储到了描述符实例中，那么随着描述符实例的共享，一方面设值方法会失效，存储在描述符实例中的属性值也会被共享。


下述代码测试了上述分析，在下述实现中，Quantity的__set__不再改变托管类实例的被托管属性的值，而是直接修改描述符实例中的属性的值。实验结果如下（说实话比较迷惑，本书中也没有展示对应的结果）：
1. 尝试直接print被托管类实例的被托管属性的值：输出的是**描述符实例**
2. 尝试print托管类实例的类属性的描述符实例中的属性值：输出的是存储在描述符实例中的值，并且多个托管类实例相互影响

In [2]:
class Quantity:

    def __init__(self, storage_name):
        self.storage_name = storage_name
        self.__dict__[storage_name] = 0
    
    def __set__(self, instance, value):
        if value > 0:
            self.__dict__[self.storage_name] = value
        else:
            raise ValueError("value must be > 0")


class LineItem:

    weight = Quantity("weight")
    price = Quantity("price")

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


test_item1 = LineItem("test1", 10, 1)
print(test_item1.weight)
print(test_item1.__class__.weight.weight)
test_item2 = LineItem("test2", 5, 1)
print(test_item1.__class__.weight.weight)

10
5


### 自动获取属性名称

上述定义中需要手动设定传入每一个描述符实例的属性名称，这样不仅不方便批量设值，还容易出错（ctrl c + ctrl v直接为所有属性设定了具有相同属性名称的描述符实例hhh）

本章介绍了一种自动设定属性名称的方法。说白了，传入描述符实例的属性名称无需和托管类中对应的属性名称一致，仅需要能够区分不同的属性即可。那么完全可以借鉴上一章中利用@property和@attr.setter定义读值以及设值的思路，即创建一个新属性用于存储值，然后在进行读值和设值时在这个新属性上进行。

基于上述分析，有如下的Quantity定义

In [5]:
class Quantity:
    _counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls._counter
        self.storage_name = "_{}#{}".format(prefix, index)
        cls._counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError("value must be > 0")


class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


test_item1 = LineItem("test1", 10, 1)
print(test_item1.weight)

10


上述实现中，值得注意的是__get__要求传入三个参数，分别为描述符实例、托管类实例以及托管类引用，最后这个托管类引用用于通过描述符获取类属性

## 基于模板方法的属性验证类层次结构设计

上述Quantity用于验证weight和price的设值是否符合要求，由于这两个属性的设值要求相同，因此可以使用同一个描述符类。现在，若想对description进行验证，在上述代码的基础上，有必要创建新描述符类。

考虑到这两个描述符类仅有__set__部分的验证逻辑不同，完全可以对上述的属性验证进行进一步的抽象。

具体来说，创建一个抽象类Validated，该类实现了__set__方法并且定义了validate方法，__set__方法通过调用validate方法对属性的设值进行验证；Validated类要求其具体子类实现对应的validate方法。

若存在另一个和抽象类Validated同层级的抽象类，例如该类不是在设值时进行检查，而是在读值时进行操作，则可以对Validated进行进一步的抽象。假设更高层次的抽象类为AutoStorage，该类用于自动存储以及管理属性，因此具有上述描述的_counter和storage_name两个属性，但是对设值以及读值方法仅是提供了接口。

根据上述分析可以有下述的层次结构：

1. AutoStorage：最高层次的抽象，提供了自动存储以及管理属性必要的功能，并且提供了设值以及读值两个功能的接口
2. Validated：低一层次的抽象，在AutoStorage的基础上定义了抽象方法validate，并且设定了一个相对具体的算法：在__set__中调用validate以执行设值检验
3. Quantity 和 NonBlank：具体子类，在Validated的基础上，实现了具体的validate方法，用于执行不同的设值检验功能

**模板方法**：
一个模板方法用一些抽象的操作定义一个算法，而子类将重定义这些操作以提供具体的行为

基于上述分析，有如下的实现：

In [7]:
import abc

class AutoStorage:
    """
    最高层次的抽象
    仅实现了自动存储和管理属性的必要属性以及接口
    """

    _counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls._counter
        self.storage_name = "_{}#{}".format(prefix, index)
        cls._counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


class Validated(abc.ABC, AutoStorage):
    """
    低一层次的抽象
    为那些需要检验设值的类提供统一接口
    """
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        """
        return validated value or rasie ValueError
        """


class Quantity(Validated):
    """
    Validated的具体子类
    用于检验设值是否大于0
    """

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("value must be > 0")
        return value


class NonBlank(Validated):
    """
    Validated的具体子类
    用于检验字符串是否为空
    """

    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError("value cannot be empty or blank")
        return value

## 覆盖型描述符以及非覆盖型描述符

## 类方法即描述符

## 描述符用法建议

## 描述符的文档字符串以及覆盖删除等操作

## 总结